In [1]:
import sys
sys.path.append('..')

In [2]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import numpy as np
from itertools import combinations
from typing import Callable 

In [3]:
relations_matrix = np.loadtxt('seating_data.csv', delimiter=',', skiprows=1)
relations_matrix = relations_matrix[ : , 1:]


In [17]:
class WSSolutionP6(Solution):
    def __init__(
        self,
        relations_matrix,
        mutation_function,  
        crossover_function,  
        repr=None,
        nr_of_tables=8
    ):
        self.relations_matrix = relations_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relations_matrix) // nr_of_tables

        self.mutation_function = mutation_function
        self.crossover_function = crossover_function
        super().__init__(repr=repr)
    
    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relations_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                while any(idx in table for table in tables):
                    idx = random.randint(0, len(self.relations_matrix) - 1)
                tables[i].append(idx)
        
        return tables

    def fitness(self):
        total_happiness = 0
        for i, row in enumerate(self.repr):
            table_happiness = 0
            combs = list(combinations(row, 2))
            pair_scores = [self.relations_matrix[a, b] for a, b in combs]
            table_happiness = sum(s for s in pair_scores)
            total_happiness += table_happiness
        return total_happiness

    def mutation(self, mut_prob):
        # Apply mutation function to representation
        new_repr = self.mutation_function(self.repr, mut_prob)
        # Create and return individual with mutated representation
        return WSSolutionP6(
           relations_matrix=self.relations_matrix,
           mutation_function=self.mutation_function,
           crossover_function=self.crossover_function,
           repr=new_repr
       )


    def crossover(self, other_solution):
        # Apply crossover function to self representation and other solution representation
        offspring1_repr, offspring2_repr = self.crossover_function(self.repr, other_solution.repr)

        # Create and return offspring with new representations
        return (
            WSSolutionP6(
                relations_matrix=self.relations_matrix,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring1_repr
            ),
            WSSolutionP6(
                relations_matrix=self.relations_matrix,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring2_repr
            )
        )


In [18]:
def get_best_ind(population: list[Solution], maximization: bool):
    fitness_list = [ind.fitness() for ind in population]
    if maximization:
        return population[fitness_list.index(max(fitness_list))]
    else:
        return population[fitness_list.index(min(fitness_list))]

def genetic_algorithmP6(
    initial_population: list[Solution],
    max_gen: int,
    selection_algorithm: Callable,
    maximization: bool = False,
    xo_prob: float = 0.9,
    mut_prob: float = 0.1,
    elitism: bool = True,
    verbose: bool = False,
):
    # 1. Initialize a population with N individuals
    population = initial_population

    # 2. Repeat until termination condition
    for gen in range(1, max_gen + 1):
        if verbose:
            print(f'-------------- Generation: {gen} --------------')

        # 2.1. Create an empty population P'
        new_population = []

        # 2.2. If using elitism, insert best individual from P into P'
        if elitism:
            new_population.append(deepcopy(get_best_ind(initial_population, maximization)))
        
        # 2.3. Repeat until P' contains N individuals
        while len(new_population) < len(population):
            # 2.3.1. Choose 2 individuals from P using a selection algorithm
            first_ind = selection_algorithm(population, maximization)
            second_ind = selection_algorithm(population, maximization)

            if verbose:
                print(f'Selected individuals:\n{first_ind}\n{second_ind}')

            # 2.3.2. Choose an operator between crossover and replication
            # 2.3.3. Apply the operator to generate the offspring
            if random.random() < xo_prob:
                offspring1, offspring2 = first_ind.crossover(second_ind)
                if verbose:
                    print(f'Applied crossover')
            else:
                offspring1, offspring2 = deepcopy(first_ind), deepcopy(second_ind)
                if verbose:
                    print(f'Applied replication')
            
            if verbose:
                print(f'Offspring:\n{offspring1}\n{offspring2}')
            
            # 2.3.4. Apply mutation to the offspring
            first_new_ind = offspring1.mutation(mut_prob)
            # 2.3.5. Insert the mutated individuals into P'
            new_population.append(first_new_ind)

            if verbose:
                print(f'First mutated individual: {first_new_ind}')
            
            if len(new_population) < len(population):
                second_new_ind = offspring2.mutation(mut_prob)
                new_population.append(second_new_ind)
                if verbose:
                    print(f'Second mutated individual: {first_new_ind}')
        
        # 2.4. Replace P with P'
        population = new_population

        if verbose:
            print(f'Final best individual in generation: {get_best_ind(population, maximization)}')

    # 3. Return the best individual in P
    return get_best_ind(population, maximization)
    

def fitness_proportionate_selection(population: list[Solution], maximization: bool):
    total_fitness = sum([ind.fitness() for ind in population])

    if maximization:
        fitness_values = [ind.fitness() for ind in population]
    else:
        # Minimization: Use the inverse of the fitness value
        # Lower fitness should have higher probability of being selected
        fitness_values = [1 / ind.fitness() for ind in population]

    total_fitness = sum(fitness_values)
    # Generate random number between 0 and total
    random_nr = random.uniform(0, total_fitness)
    box_boundary = 0
    # For each individual, check if random number is inside the individual's "box"
    for ind_idx, ind in enumerate(population):
        box_boundary += fitness_values[ind_idx]
        if random_nr <= box_boundary:
            return deepcopy(ind)

def binary_standard_mutation(representation: str | list, mut_prob):
    
    # Initialize new representation as a copy of current representation
    new_representation = deepcopy(representation)

    if random.random() <= mut_prob:
        # Strings are not mutable. Let's convert temporarily to a list
        if isinstance(representation, str):
            new_representation = list(new_representation)

        for char_ix, char in enumerate(representation):
            if char == "1":
                new_representation[char_ix] = "0"
            elif char == 1:
                new_representation[char_ix] = 0
            elif char == "0":
                new_representation[char_ix] = "1"
            elif char == 0:
                new_representation[char_ix] = 1
            else:
                raise ValueError(f"Invalid character {char}. Can not apply binary standard mutation")
    
        # If representation was a string, convert list back to string
        if isinstance(representation, str):
            new_representation = "".join(new_representation)

    return new_representation

def swap_mutation(representation, mut_prob):
    
    # Strings are not mutable. Let's convert to list first
    if isinstance(representation, str):
        new_representation = deepcopy(list(representation))
    elif isinstance(representation, list):
        new_representation = deepcopy(representation)

    if random.random() <= mut_prob:
        first_idx = random.randint(0, len(representation) - 1)

        # To guarantee we select two different positions
        second_idx = first_idx
        while second_idx == first_idx:
            second_idx = random.randint(0, len(representation) - 1)

        new_representation[first_idx] = representation[second_idx]
        new_representation[second_idx] = representation[first_idx]


    # If representation was a string, convert list back to string
    if isinstance(representation, str):
        new_representation = "".join(new_representation)
    
    return new_representation

def standard_crossover(parent1_repr, parent2_repr):
    
    # Choose random crossover point
    xo_point = random.randint(1, len(parent1_repr) - 1)

    offspring1_repr = parent1_repr[:xo_point] + parent2_repr[xo_point:]
    offspring2_repr = parent2_repr[:xo_point] + parent1_repr[xo_point:]

    return offspring1_repr, offspring2_repr

In [19]:
POP_SIZE = 50
initial_population = [WSSolutionP6(relations_matrix=relations_matrix,  mutation_function=swap_mutation, crossover_function = standard_crossover) for _ in range(POP_SIZE)]

best_solution = genetic_algorithmP6(
    initial_population=initial_population,
    selection_algorithm=fitness_proportionate_selection,
    max_gen=50,
    maximization=True,
    verbose=False,
    elitism=False,
)

print("Best solution:", best_solution)
print("Fitness:", best_solution.fitness())

KeyboardInterrupt: 